# **수상작 리뷰**

## 데이터셋 정보

주제 : 최적 비료(fertilizer) 예측.

데이터 구성
* 8개의 범주형(feature) : 토양 종류, 기후 구역, 농업 방식 등을 암시하는 synthetic
* target : 비료 클래스 (0~6, 총 7개 클래스)
* train/test : 플레이그라운드의 특성상 원본 데이터에서 합성(sampled) 된 데이터셋

## 전처리

(1) 조합 특징 생성
* 8개 feature
* 모든 pair → 28개
* 모든 triple → 56개
* 모든 quadruple → 70개
* 총 162개의 조합 feature를 생성 → 범주형 feature space를 폭발적으로 확장하는 전략.

(2) Target Encoding
특히 cuML Target Encoder로:*
* 162개 조합 feature × 7개의 이진 타겟(y==0? … y==6?) = 1134개 target-encoded 특징
* 원본 feature 8개에 대한 target encoding 추가 → total 2268개의 encoded feature 생성

(3) External Original Data 활용

원본 데이터셋을 활용하면 강력한 성능 향상이 발생함.

활용 방식:
* 행(Row) 추가
* 열(Column) 생성 ← 이번 솔루션의 핵심 차별점

## 모델링

1단계 모델들 (Level 1)
* XGBoost 여러 버전 (depth 4, 10, 18 등)
* CatBoost
* Neural Networks
* cuML Linear Regression
* Repeated K-Fold XGB
* External-based 모델들
* Pseudo-labeled 모델들

Level 2 Stacking
* NN 기반 Stacker (Categorical CE Loss, calibrated probability 자동 개선)
* XGBoost 기반 Stacker (multi:softprob objective)
* 두 스태커 결과 평균 → 성능 상승

Repeated KFold Ensembling
* 같은 모델을 seed만 바꿔 수십~수백 번 훈련
* 확률 평균 후 MAP@3 계산 → 점수 상승
("randomness smoothing" 효과)

Hill Climbing Ensemble
* 각 모델에 weight를 조정하며 성능이 최고가 되는 조합 탐색
* GPU 힐클라임이기 때문에 가능한 강력한 brute-force search

# 차별점 및 배울점

feature engineering을 8개에서 2268개로 확장하였다. 다들 row augmentation만 하는 가운데, 원본 데이터를 열로 활용하는 차별화된 기법이 인상적이었다. 이 프로젝트를 통해 스태킹은 확률 보정에 정말 유리함을 배울 수 있었다.  

